In [ ]:
# ==============================================================================
# CORAL CONTACT SIMULATION AUTOMATION SCRIPT
# ==============================================================================
# Description: Automates the setup, solution, and result export for a contact 
#              simulation between grippers and a coral geometry.
#              Includes advanced Named Selection logic (proximity, facing-down detection).
# Source:
# ==============================================================================

import os
import time
import datetime
import math
import csv
from pathlib import Path

# --- Ansys Mechanical Dependencies ---
from ansys.mechanical.core import App
from Ansys.Mechanical.DataModel.Enums import *
from Ansys.ACT.Mechanical.Utilities import GeometryImportPreferences

# --- Visualization Dependencies ---
from matplotlib import image as mpimg
from matplotlib import pyplot as plt

# ==============================================================================
# 1. SIMULATION PARAMETERS (CONSTANTS)
# ==============================================================================
# Geometry Dimensions
HOLE_RADIUS = 2.0 

# Geometry Import Indices (0-based index of bodies in the tree)
# Based on your script: Gripper2=0, Gripper1=1, Coral=2
IDX_GRIPPER_2 = 0
IDX_GRIPPER_1 = 1
IDX_CORAL = 2

# Named Selection (NS) Tags
NS_NAME_GRIPPER_1_HOLE = "NS_Hole_Gripper1"
NS_NAME_GRIPPER_2_HOLE = "NS_Hole_Gripper2"
NS_NAME_CONTACT_G1 = "NS_Contact_Gripper1_Inner"
NS_NAME_CONTACT_G2 = "NS_Contact_Gripper2_Inner"
NS_NAME_CORAL_BODY = "NS_Coral_Full_Body"
NS_NAME_CORAL_CONTACT_1 = "NS_Contact_Coral_Zone1"
NS_NAME_CORAL_CONTACT_2 = "NS_Contact_Coral_Zone2"
NS_NAME_CORAL_BOTTOM = "NS_Coral_Bottom_Faces"

# Initialize App
app = App()
app.update_globals(globals())
print(f"Connected to: {app}")

# Start Timer
start_time = time.time()

In [ ]:
def create_hole_named_selection(body_index, ns_name):
    """
    Identifies a cylindrical hole face in a specific body based on HOLE_RADIUS.
    """
    all_bodies = Model.GetChildren(DataModelObjectCategory.Body, True)
    if len(all_bodies) > body_index:
        body = all_bodies[body_index]
        geo_body = body.GetGeoBody()
        found_faces = []

        for face in geo_body.Faces:
            try:
                if math.isclose(face.Radius, HOLE_RADIUS, rel_tol=1e-5):
                    found_faces.append(face)
            except:
                pass

        if found_faces:
            ns = Model.AddNamedSelection()
            ns.Name = ns_name
            selection = ExtAPI.SelectionManager.CreateSelectionInfo(SelectionTypeEnum.GeometryEntities)
            selection.Ids = [f.Id for f in found_faces]
            ns.Location = selection
            print(f"-> NS '{ns_name}' created ({len(found_faces)} faces).")
        else:
            print(f"WARNING: No hole found in body '{body.Name}'.")

def create_ns_by_proximity(seeker_body_idx, target_body_idx, ns_name):
    """
    Finds the face on the 'seeker' body closest to the 'target' body's centroid.
    """
    all_bodies = Model.GetChildren(DataModelObjectCategory.Body, True)
    seeker = all_bodies[seeker_body_idx]
    target = all_bodies[target_body_idx]
    target_centroid = target.GetGeoBody().Centroid

    closest_face = None
    min_dist = float('inf')

    for face in seeker.GetGeoBody().Faces:
        c = face.Centroid
        dist = math.sqrt((c[0]-target_centroid[0])**2 + (c[1]-target_centroid[1])**2 + (c[2]-target_centroid[2])**2)
        if dist < min_dist:
            min_dist = dist
            closest_face = face

    if closest_face:
        ns = Model.AddNamedSelection()
        ns.Name = ns_name
        sel = ExtAPI.SelectionManager.CreateSelectionInfo(SelectionTypeEnum.GeometryEntities)
        sel.Ids = [closest_face.Id]
        ns.Location = sel
        print(f"-> NS '{ns_name}' created by proximity ({min_dist:.2f} mm).")

def create_ns_full_body(body_index, ns_name):
    """Creates a Named Selection containing the entire body."""
    all_bodies = Model.GetChildren(DataModelObjectCategory.Body, True)
    if len(all_bodies) > body_index:
        body = all_bodies[body_index]
        ns = Model.AddNamedSelection()
        ns.Name = ns_name
        geo_body = body.GetGeoBody()
        sel = ExtAPI.SelectionManager.CreateSelectionInfo(SelectionTypeEnum.GeometryEntities)
        sel.Ids = [geo_body.Id]
        ns.Location = sel
        print(f"-> NS '{ns_name}' created (Full Body).")
        return ns
    return None

def create_ns_by_distance(source_body_idx, target_ns_name, threshold_mm, output_ns_name):
    """
    Selects faces on 'source_body' that are within 'threshold_mm' of the 
    centroid of the geometry defined in 'target_ns_name'.
    """
    all_bodies = Model.GetChildren(DataModelObjectCategory.Body, True)
    source_body = all_bodies[source_body_idx]
    
    # Find Target NS
    target_ns = next((ns for ns in Model.NamedSelections.Children if ns.Name == target_ns_name), None)
    if not target_ns:
        print(f"ERROR: Target NS '{target_ns_name}' not found.")
        return

    # Get Target Centroid (assuming single entity in NS for simplicity, usually a face)
    # Note: Accessing underlying geometry requires handling the wrapper
    target_entity = target_ns.Location.Entities[0] 
    # Logic to find the geometry object from the ID to get Centroid
    target_geo_body = next((b for b in all_bodies if b.GetGeoBody().Id == target_entity.Body.Id), None)
    target_face = next((f for f in target_geo_body.GetGeoBody().Faces if f.Id == target_entity.Id), None)
    
    if not target_face: return
    target_centroid = target_face.Centroid

    # Filter Faces
    faces_in_range = []
    for face in source_body.GetGeoBody().Faces:
        c = face.Centroid
        dist = math.sqrt((c[0]-target_centroid[0])**2 + (c[1]-target_centroid[1])**2 + (c[2]-target_centroid[2])**2)
        if dist <= threshold_mm:
            faces_in_range.append(face.Id)

    if faces_in_range:
        ns = Model.AddNamedSelection()
        ns.Name = output_ns_name
        sel = ExtAPI.SelectionManager.CreateSelectionInfo(SelectionTypeEnum.GeometryEntities)
        sel.Ids = faces_in_range
        ns.Location = sel
        print(f"-> NS '{output_ns_name}' created ({len(faces_in_range)} faces within {threshold_mm}mm).")

def create_ns_faces_facing_down(body_index, ns_name, tolerance=0.5):
    """
    Selects faces positioned at the lowest Z-coordinate (the 'bottom' of the object).
    """
    all_bodies = Model.GetChildren(DataModelObjectCategory.Body, True)
    body = all_bodies[body_index]
    geo_body = body.GetGeoBody()

    # 1. Find Global Min Z
    z_coords = [f.Centroid[2] for f in geo_body.Faces]
    min_z = min(z_coords)

    # 2. Select faces near Min Z
    bottom_faces = []
    for face in geo_body.Faces:
        if abs(face.Centroid[2] - min_z) <= tolerance:
            bottom_faces.append(face.Id)

    if bottom_faces:
        ns = Model.AddNamedSelection()
        ns.Name = ns_name
        sel = ExtAPI.SelectionManager.CreateSelectionInfo(SelectionTypeEnum.GeometryEntities)
        sel.Ids = bottom_faces
        ns.Location = sel
        print(f"-> NS '{ns_name}' created ({len(bottom_faces)} bottom faces found).")

def apply_fixed_support(ns_name):
    ns = next((n for n in Model.NamedSelections.Children if n.Name == ns_name), None)
    if ns:
        analysis = Model.Analyses[0]
        fix = analysis.AddFixedSupport()
        sel = ExtAPI.SelectionManager.CreateSelectionInfo(SelectionTypeEnum.GeometryEntities)
        sel.Ids = ns.Location.Ids
        fix.Location = sel

def apply_moment_load(ns_name, magnitude, direction):
    ns = next((n for n in Model.NamedSelections.Children if n.Name == ns_name), None)
    if ns:
        analysis = Model.Analyses[0]
        load = analysis.AddMoment()
        sel = ExtAPI.SelectionManager.CreateSelectionInfo(SelectionTypeEnum.GeometryEntities)
        sel.Ids = ns.Location.Ids
        load.Location = sel
        load.DefineBy = LoadDefineBy.Vector
        load.Magnitude.Output.SetDiscreteValue(0, Quantity(magnitude, "N m"))
        load.Direction = Ansys.ACT.Math.Vector3D(direction[0], direction[1], direction[2])

def progress_bar(current, total, msg=""):
    percent = int((current / total) * 100)
    bar = "█" * (percent // 5) + "_" * (20 - percent // 5)
    print(f"[{bar}] {percent}% -> {msg}")

In [ ]:
def display_image(image_path):
    try:
        plt.figure(figsize=(16, 9))
        plt.imshow(mpimg.imread(image_path))
        plt.axis("off")
        plt.show()
        plt.close()
    except Exception:
        pass

def generate_and_export_simulation_views(output_folder, export_settings, display_func):
    """
    Generates simulation views customized for the Coral geometry.
    """
    print("\n--- STARTING VISUALIZATION EXPORT ---")
    try:
        analysis = Model.Analyses[0]
        solution = analysis.Solution
        all_bodies = Model.GetChildren(DataModelObjectCategory.Body, True)
        all_ns = Model.NamedSelections.Children
    except:
        return

    def isolate_and_capture(ns_names, filename, rot_y=60, rot_x=-30):
        target_bodies = []
        ns_to_activate = []
        try:
            for name in ns_names:
                ns_obj = next((ns for ns in all_ns if ns.Name == name), None)
                if ns_obj:
                    ns_to_activate.append(ns_obj)
                    # Simple body finding logic based on NS entity
                    if ns_obj.Location.Entities:
                        b_id = ns_obj.Location.Entities[0].Body.Id
                        body = next((b for b in all_bodies if b.GetGeoBody().Id == b_id), None)
                        if body and body not in target_bodies: target_bodies.append(body)
            
            if not target_bodies: target_bodies = all_bodies
            for b in all_bodies: b.Visible = False
            for b in target_bodies: b.Visible = True
            if ns_to_activate: Tree.Activate(ns_to_activate)

            Graphics.Camera.SetFit()
            Graphics.Camera.Rotate(rot_y, CameraAxisType.ScreenY)
            Graphics.Camera.Rotate(rot_x, CameraAxisType.ScreenX)
            
            fpath = os.path.join(output_folder, filename)
            Graphics.ExportImage(fpath, GraphicsImageExportFormat.PNG, export_settings)
            display_func(fpath)
        except Exception as e:
            print(f"Error capturing {filename}: {e}")
        finally:
            for b in all_bodies: b.Visible = True
            Tree.Activate([Model.Geometry])

    # 1. Holes
    isolate_and_capture([NS_NAME_GRIPPER_1_HOLE, NS_NAME_GRIPPER_2_HOLE], "View_Holes.png")

    # 2. Contact Surfaces (Grippers + Coral)
    isolate_and_capture(
        [NS_NAME_CONTACT_G1, NS_NAME_CONTACT_G2, NS_NAME_CORAL_CONTACT_1, NS_NAME_CORAL_CONTACT_2], 
        "View_Contacts.png"
    )

    # 3. Bottom Support
    isolate_and_capture([NS_NAME_CORAL_BOTTOM], "View_Support.png")

    # 4. Results (Deformation)
    try:
        sol_def = solution.Children[1]
        sol_def.EvaluateAllResults()
        Tree.Activate([sol_def])
        Graphics.Camera.SetFit()
        Graphics.Camera.Rotate(30, CameraAxisType.ScreenX)
        fpath = os.path.join(output_folder, "Result_Deformation.png")
        Graphics.ExportImage(fpath, GraphicsImageExportFormat.PNG, export_settings)
        display_func(fpath)
    except:
        pass

    Tree.Activate([Model.Geometry])

In [ ]:
# Graphics Settings
image_export_format = GraphicsImageExportFormat.PNG
settings_720p = Ansys.Mechanical.Graphics.GraphicsImageExportSettings()
settings_720p.Resolution = GraphicsResolutionType.EnhancedResolution
settings_720p.Background = GraphicsBackgroundType.White
settings_720p.Width = 1280
settings_720p.Height = 720
settings_720p.CurrentGraphicsDisplay = False

def run_simulation_workflow(cad_file_path, base_target_folder):
    """
    Main execution logic for Coral Simulation.
    """
    TOTAL_STEPS = 12
    step = 0
    file_name = os.path.basename(cad_file_path)
    print(f"\n{'='*60}")
    print(f"| 🚀 STARTING SIMULATION: {file_name}")
    print(f"{'='*60}")

    # 1. Initialize
    app.new()
    app.update_globals(globals())
    step += 1; progress_bar(step, TOTAL_STEPS, "Importing Geometry")
    
    geom_import = Model.GeometryImportGroup.AddGeometryImport()
    prefs = GeometryImportPreferences()
    prefs.ProcessNamedSelections = True
    geom_import.Import(cad_file_path, GeometryImportPreference.Format.Automatic, prefs)
    ExtAPI.Application.ActiveUnitSystem = MechanicalUnitSystem.StandardMKS

    # 2. Basic Named Selections
    step += 1; progress_bar(step, TOTAL_STEPS, "Creating Gripper NS")
    create_hole_named_selection(IDX_GRIPPER_1, NS_NAME_GRIPPER_1_HOLE)
    create_hole_named_selection(IDX_GRIPPER_2, NS_NAME_GRIPPER_2_HOLE)
    
    create_ns_by_proximity(IDX_GRIPPER_1, IDX_CORAL, NS_NAME_CONTACT_G1)
    create_ns_by_proximity(IDX_GRIPPER_2, IDX_CORAL, NS_NAME_CONTACT_G2)

    # 3. Advanced Coral NS
    step += 1; progress_bar(step, TOTAL_STEPS, "Creating Coral NS")
    create_ns_full_body(IDX_CORAL, NS_NAME_CORAL_BODY)
    
    # Create contact zones on Coral based on distance from Gripper contact faces
    create_ns_by_distance(IDX_CORAL, NS_NAME_CONTACT_G1, 20.0, NS_NAME_CORAL_CONTACT_1)
    create_ns_by_distance(IDX_CORAL, NS_NAME_CONTACT_G2, 20.0, NS_NAME_CORAL_CONTACT_2)

    # 4. Joints
    step += 1; progress_bar(step, TOTAL_STEPS, "Creating Joints")
    conns = Model.Connections
    grp = conns.AddConnectionGroup()
    for i, ns in enumerate([NS_NAME_GRIPPER_1_HOLE, NS_NAME_GRIPPER_2_HOLE]):
        ns_obj = next((n for n in Model.NamedSelections.Children if n.Name == ns), None)
        if ns_obj:
            joint = grp.AddJoint()
            joint.ConnectionType = JointScopingType.BodyToGround
            joint.Type = JointType.Revolute
            joint.MobileLocation = ns_obj
            joint.CoordinateSystem = Model.CoordinateSystems.Children[0]
            joint.PrincipalAxisZ = 1
            if i == 1: joint.MobileRelaxationMethod = False # Default

    # 5. Contacts
    step += 1; progress_bar(step, TOTAL_STEPS, "Creating Contacts")
    cont_grp = conns.AddConnectionGroup()
    ns_coral_full = next((n for n in Model.NamedSelections.Children if n.Name == NS_NAME_CORAL_BODY), None)
    
    for ns_src_name in [NS_NAME_CONTACT_G1, NS_NAME_CONTACT_G2]:
        ns_src = next((n for n in Model.NamedSelections.Children if n.Name == ns_src_name), None)
        if ns_src and ns_coral_full:
            c = cont_grp.AddContactRegion()
            c.SourceLocation = ns_src.Location
            c.TargetLocation = ns_coral_full.Location
            c.ContactType = ContactType.Frictionless

    # 6. Contact Tool & Mesh
    step += 1; progress_bar(step, TOTAL_STEPS, "Meshing")
    conns.AddContactTool()
    conns.GenerateInitialContactResults()
    
    Model.AddTransientStructuralAnalysis()
    mesh = Model.Mesh
    mesh.ElementOrder = ElementOrder.Linear
    mesh.ElementSize = Quantity("2 [mm]")
    
    # Refine contacts
    ref = mesh.AddRefinement()
    ref.NumberOfRefinements = 2
    ids = []
    for name in [NS_NAME_CONTACT_G1, NS_NAME_CONTACT_G2, NS_NAME_CORAL_CONTACT_1, NS_NAME_CORAL_CONTACT_2]:
        n = next((x for x in Model.NamedSelections.Children if x.Name == name), None)
        if n: ids.extend(list(n.Location.Ids))
    
    if ids:
        sel = ExtAPI.SelectionManager.CreateSelectionInfo(SelectionTypeEnum.GeometryEntities)
        sel.Ids = ids
        ref.Location = sel
    
    mesh.GenerateMesh()

    # 7. Analysis Settings
    step += 1; progress_bar(step, TOTAL_STEPS, "Analysis Settings")
    analysis = Model.Analyses[0]
    sett = analysis.AnalysisSettings
    sett.NumberOfSteps = 1
    sett.StepEndTime = Quantity("1 [s]")
    sett.LargeDeflection = True
    sett.AutomaticTimeStepping = AutomaticTimeStepping.On
    sett.InitialTimeStep = Quantity("0.01 [s]")
    sett.MinimumTimeStep = Quantity("0.001 [s]")
    sett.MaximumTimeStep = Quantity("0.01 [s]")
    sett.NodalForces = OutputControlsNodalForcesType.Yes

    # 8. Boundary Conditions (Support & Load)
    step += 1; progress_bar(step, TOTAL_STEPS, "Loads & Supports")
    
    # Identify bottom faces for support
    create_ns_faces_facing_down(IDX_CORAL, NS_NAME_CORAL_BOTTOM, tolerance=0.6)
    apply_fixed_support(NS_NAME_CORAL_BOTTOM)
    
    # Apply moments
    apply_moment_load(NS_NAME_GRIPPER_1_HOLE, -1, [0,0,1])
    apply_moment_load(NS_NAME_GRIPPER_2_HOLE, 1, [0,0,1])

    # 9. Results Setup
    step += 1; progress_bar(step, TOTAL_STEPS, "Configuring Results")
    sol = analysis.Solution
    sol.AddTotalDeformation()
    ct = sol.AddContactTool()
    ct.AddPressure()
    
    for i, cr in enumerate(cont_grp.Children):
        probe = sol.AddForceReaction()
        probe.Name = f"Force_Reaction_Gripper_{i+1}"
        probe.LocationMethod = LocationDefinitionMethod.ContactRegion
        probe.ContactRegionSelection = cr

    # 10. Solve
    step += 1; progress_bar(step, TOTAL_STEPS, "Solving...")
    analysis.Solve()

    # 11. Export
    step += 1; progress_bar(step, TOTAL_STEPS, "Exporting Data")
    file_base = os.path.splitext(file_name)[0]
    out_dir = os.path.join(base_target_folder, file_base)
    if not os.path.exists(out_dir): os.makedirs(out_dir)
    
    generate_and_export_simulation_views(out_dir, settings_720p, display_image)
    
    # Data extraction
    p1 = sol.Children[3]; p1.EvaluateAllResults()
    p2 = sol.Children[4]; p2.EvaluateAllResults()
    press = sol.Children[2].Children[1]; press.EvaluateAllResults()
    
    res = {
        'FileName': file_base,
        'Fx1': p1.XAxis.Value, 'Fy1': p1.YAxis.Value, 'Ftot1': p1.Total.Value,
        'Fx2': p2.XAxis.Value, 'Fy2': p2.YAxis.Value, 'Ftot2': p2.Total.Value,
        'Pressure_Max': press.Maximum.Value,
        'Pressure_Avg': press.Average.Value
    }
    
    # Save text summary
    with open(os.path.join(out_dir, f"{file_base}_RESULTS.txt"), "w") as f:
        f.write(str(res))
        
    # Save project
    app.save(os.path.join(base_target_folder, f"{file_base}.mechdb"))
    
    step += 1; progress_bar(step, TOTAL_STEPS, "Done")
    return res

In [ ]:
def run_batch_simulation(directory):
    directory = Path(directory)
    cad_files = [str(p) for p in directory.iterdir() if p.suffix.lower() in ['.step', '.stp']]
    
    if not cad_files:
        print("No CAD files found.")
        return

    print(f"Found {len(cad_files)} files.")
    all_results = []
    
    for cad in cad_files:
        try:
            r = run_simulation_workflow(cad, str(directory))
            all_results.append(r)
        except Exception as e:
            print(f"ERROR processing {cad}: {e}")
            
    if all_results:
        csv_path = directory / f"MASTER_RESULTS_{datetime.datetime.now().strftime('%Y%m%d')}.csv"
        with open(csv_path, 'w', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=all_results[0].keys())
            writer.writeheader()
            writer.writerows(all_results)
        print(f"Batch completed. Saved: {csv_path}")

# --- RUN ---
INPUT_DIRECTORY = r"C:\\" # Update this path
if os.path.exists(INPUT_DIRECTORY):
    run_batch_simulation(INPUT_DIRECTORY)
else:
    print("Directory not found.")